In [2]:
import pandas as pd
import numpy as np
import os
import image_check as img_check

# import embedder as emb

/home/ale/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/ale/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
csv_path_sold_df = "/home/ale/Desktop/Vinted-Web-Scraper/sold_items.csv"
sold_items_df = pd.read_csv(csv_path_sold_df)

In [11]:
csv_path = "/home/ale/Desktop/Vinted-Web-Scraper/big_csv/big_csv_prova_merge.csv"
big_df = pd.read_csv(csv_path)

# Fix period in prices


In [ ]:
# def fix_price(price):
#     if isinstance(price, str) and '.' not in price and price.isdigit():
#         return f"{price[:-2]}.{price[-2:]}"  # Insert period before the last two digits
#     return price

# # Apply the function to the 'Price' column
# big_df['Price'] = big_df['Price'].apply(fix_price)

# Drop columns

In [ ]:
columns_to_drop = ["Page", "SearchCount", "Condition", "LastCheck", "Dataid", "Link", "" ]
big_df = big_df.drop(columns=columns_to_drop)

# Parse size and price to float

In [11]:
def float_size(value):
    if isinstance(value, int) or isinstance(value, str):
        return float(value)
    return value

def int_likes(value):
    if isinstance(value, str):
        return int(value)
    return value

big_df['Size'] = pd.to_numeric(big_df['Size'], errors='coerce')
big_df = big_df.dropna(subset=['Size'])

big_df['Price'] = pd.to_numeric(big_df['Price'], errors='coerce')
big_df = big_df.dropna(subset=['Price'])

big_df["Likes"] = big_df["Likes"].apply(int_likes)

big_df.to_csv("/home/ale/Desktop/Vinted-Web-Scraper/big_csv.csv")

In [13]:
sold_items_df['Size'] = pd.to_numeric(sold_items_df['Size'], errors='coerce')
sold_items_df = sold_items_df.dropna(subset=['Size'])

sold_items_df['Price'] = pd.to_numeric(sold_items_df['Price'], errors='coerce')
sold_items_df = sold_items_df.dropna(subset=['Price'])

sold_items_df["Likes"] = sold_items_df["Likes"].apply(int_likes)

sold_items_df.to_csv("/home/ale/Desktop/Vinted-Web-Scraper/sold_items.csv")


In [10]:
print(type(big_df.loc[7220, "Price"]))

<class 'numpy.float64'>


# Remove sold items from big csv

In [11]:
sold_csv_path = "/home/ale/Desktop/Vinted-Web-Scraper/sold_items.csv"
sold_df = pd.read_csv(sold_csv_path)


df_result = pd.merge(big_df, sold_df, on="Dataid", how='left', indicator=True).query('_merge == "left_only"').drop(columns='_merge')

print(len(df_result))

55114


# Cast Market Status 

In [ ]:
big_df['MarketStatus'] = 0

# Move 'MarketStatus' column to the end
market_status = big_df.pop('MarketStatus')
big_df['MarketStatus'] = market_status

In [ ]:
sold_descriptions = df_final[df_final['MarketStatus'] == 1]['Description'].tolist()
not_sold_descriptions = df_final[df_final['MarketStatus'] == 0]['Description'].tolist()

In [ ]:
# Getting embeddings
sold_embeddings = [emb.get_embedding(sentence) for sentence in sold_descriptions]
not_sold_embeddings = [emb.get_embedding(sentence) for sentence in not_sold_descriptions]

# Calculating average locations (centroids)
sold_centroid = emb.calculate_centroid(sold_embeddings)
not_sold_centroid = emb.calculate_centroid(not_sold_embeddings)

# Calculate images stats

## Create columns for image stats

In [12]:
columns_to_check = {
    "Mean": float,
    "Standard Deviation": float,
    "Minimum": float,
    "Maximum": float,
    "Skewness": float,
    "Kurtosis": float,
    "ImageCount": int,
    "ImageStats": str
}

# Identify missing columns
missing_columns = {col: dtype for col, dtype in columns_to_check.items() if col not in big_df.columns}

# Add missing columns with specified data types
for col, dtype in missing_columns.items():
    big_df[col] = pd.Series(dtype=dtype)  # Initialize empty column with correct dtype

print(big_df.columns)

big_df.to_csv(csv_path)

Index(['Unnamed: 0', 'Title', 'Price', 'Brand', 'Size', 'Likes', 'Images',
       'Dataid', 'Downloaded', 'Mean', 'Standard Deviation', 'Minimum',
       'Maximum', 'Skewness', 'Kurtosis', 'ImageCount', 'ImageStats'],
      dtype='object')


## Populate the df

In [14]:
sold_images_folder = "/home/ale/Desktop/Vinted-Web-Scraper/sold_items_images"
big_csv_images = "/home/ale/Desktop/Vinted-Web-Scraper/big_csv_images"

def compute_image_stats(row):
    print(row["Dataid"])
    if row["ImageStats"] != "Yes":
        folder_path = os.path.join(big_csv_images, str(row["Dataid"]))
        image_stats = img_check.calculate_images_blur_stats(folder_path)

        # Return updated values
        return pd.Series({
            'Mean': image_stats["Mean"],
            'Standard Deviation': image_stats["Standard Deviation"],
            'Minimum': image_stats["Minimum"],
            'Maximum': image_stats["Maximum"],
            'Skewness': image_stats["Skewness"],
            'Kurtosis': image_stats["Kurtosis"],
            'ImageCount': image_stats["ImageCount"],
            'ImageStats': "Yes"
        })
    return row[['Mean', 'Standard Deviation', 'Minimum', 'Maximum', 'Skewness', 'Kurtosis', 'ImageCount', 'ImageStats']]

# Apply function to update the DataFrame efficiently
big_df[['Mean', 'Standard Deviation', 'Minimum', 'Maximum', 'Skewness', 'Kurtosis', 'ImageCount', 'ImageStats']] = big_df.apply(compute_image_stats, axis=1)


big_df.to_csv(csv_path)

# for index, row in sold_items_df.iterrows():
#     if row["ImageStats" != "Yes"]:
#         dataid = str(row["Dataid"])
#         folder_path = os.path.join(sold_images_folder, dataid)
#         image_stats = img_check.calculate_images_blur_stats(folder_path)
#         sold_items_df.loc[dataid, 'Mean'] = image_stats["Mean"]
#         sold_items_df.loc[dataid, 'Standard Deviation'] = image_stats["Standard Deviation"]
#         sold_items_df.loc[dataid, 'Minimum'] = image_stats["Minimum"]
#         sold_items_df.loc[dataid, 'Maximum'] = image_stats["Maximum"]
#         sold_items_df.loc[dataid, 'Skewness'] = image_stats["Skewness"]
#         sold_items_df.loc[dataid, 'Kurtosis'] = image_stats["Kurtosis"]
#         sold_items_df.loc[dataid, 'ImageCount'] = image_stats["ImageCount"]
#         sold_items_df.loc[dataid, 'ImageStats'] = "Yes"

5646585088
5646585112
5646567012
5646585017
5646584623
5646584883
5646584754
5646584175
5646584252
5646583266
5646582766
5646583076
5615694509
5646582812
5646582794
5646582623
5646582170
5646581220
5646580802
5646733525
5646733384
5646733090
5646733160
5646732600
5646731613
5646731214
5646730790
5646730470
5646730420
5646730129
5646730127
5646729574
5646729625
5646728649
5646728828
5646728476
5646727078
5646728520
5646727814
5640270752
5646644947
4155616872
5646727728
5646727580
5646727430
5646726813
5646726274
5646726261
5646726163
5646725438
5646725107
5646724998
5646724869
5646724770
5646724179
4502948622
5646724361
5646724106
5646724291
5646724172
5646724012
5646723182
5646722896
5646496211
5646745571
5643281170
5640376747
5646745227
5646744990
5646744060
5646742693
5646742356
5646742386
5646742077
5646705507
5646741874
5646741043
5646741211
5646740392
5646740469
5646740459
5646740169
5646740080
5646739839
5646702779
5646739110
5646739074
5646739028
5646737808
5646738519
5646738315

TypeError: 'NoneType' object is not subscriptable